# 5.3 控制随机性的译码策略

本章节我们会介绍文本生成策略(也称译码策略)来生成更多原始文本.首先我们简单回顾一下在本章节前面的generate_and_print_sample中使用的前面章节的generate_text_simple函数。接着我们会介绍用于优化该函数的两项技术：温度放缩（temperature scaling）和top-k采样

我们首先将模型从GPU传输回CPU，因为使用相对较小的模型进行推理不需要 GPU。然后我们将经过训练的模型放入评估模型中以关闭随机组件例如dropout：

```
model.to("cpu")
model.eval()
```

接着我们将GPTModel实例（模型）插入到generate_text_simple函数中，该函数使用LLM一次生成一个token：

```python
tokenizer = tiktoken.get_encoding("gpt2")
token_ids = generate_text_simple(
    model=model,
    idx=text_to_token_ids("Every effort moves you", tokenizer),
    max_new_tokens=25,
    context_size=GPT_CONFIG_124M["context_length"]
)
print("Output text:\n", token_ids_to_text(token_ids, tokenizer))


生成的文本如下所示：

```
Output text:
Every effort moves you know," was one of the axioms he laid down
```

正如前面在5.1.2节所述，在每个生成步骤中选出的生成的token与词汇表中所有token中的最大概率分数相对应。

这意味着在相同开头的语境中（例如“Every effort moves you”）无论我们运行generate_text_simple函数多少次，LLM都会一直生成同样的结果。

在接下来的小节中会介绍两个控制随机性和多样性的概念：温度放缩（temperature scaling）和top-k采样

## 5.3.1 温度放缩（Temperature scaling）

该部分介绍的温度放缩是一项将概率选择过程添加到下一token生成任务的技术。

以前在generate_text_simple函数中，我们总是用torch.argmax 抽取概率最高的令牌作为下一个令牌，这一过程也称为贪婪解码（greedy decoding）。为了生成更具多样性的文本，我们可以将argmax替换成从概率分布中采样的函数（在此特指LLM在每个token生成步骤中为每个词汇条目生成的概率分数）。

为了用具体的例子来阐明概率抽样，让我们用一个非常小的词汇来简要讨论下一个token的生成过程以便说明问题：

```
vocab = {
    "closer": 0,
    "every": 1,
    "effort": 2,
    "forward": 3,
    "inches": 4,
    "moves": 5,
    "pizza": 6,
    "toward": 7,
    "you": 8,
}
inverse_vocab = {v: k for k, v in vocab.items()}


下面假设LLM开头语境被设定为“every efforts moves you”，并生成如下所示的的下一token的logit：

```
next_token_logits = torch.tensor([4.51, 0.89, -1.90, 6.75, 1.63, -1.62, -1.89, 6.28, 1.79])
```

正如上一章在generate_text_simple中所讨论的，我们通过softmax函数将logits转化为概率并通过argmax函数获取与生成的token相应的token ID，然后我们可以通过逆转词汇表将其映射回文本：

```
probas = torch.softmax(next_token_logits, dim=0)
next_token_id = torch.argmax(probas).item()
print(inverse_vocab[next_token_id])


由于最大的logit值以及相应的最大softmax概率分数位于第四个位置（索引位置为3，因为Python采用0索引制），因此生成的单词是“forward”。

为了实现概率采样过程，我们现在可以将 argmax 替换为 PyTorch 中的多项式函数：

```
torch.manual_seed(123)
next_token_id = torch.multinomial(probas, num_samples=1).item()
print(inverse_vocab[next_token_id])
```

但打印输出结果仍为“forward”。为何如此？这是因为多项式函数对下一个token的采样与其概率分数成正比。换言之，“forward”仍是最具可能性的token并且大多时候都会被多项式选择，但有时也有例外。为了说明这一点，让我们实现一个重复采样1000次的函数：

```
def print_sampled_tokens(probas):
    torch.manual_seed(123)
    sample = [torch.multinomial(probas, num_samples=1).item() for i in range(1_000)]
    sampled_ids = torch.bincount(torch.tensor(sample))
    for i, freq in enumerate(sampled_ids):
        print(f"{freq} x {inverse_vocab[i]}")
    print_sampled_tokens(probas)
```


采样结果如下所示：

```
73 x closer 
0 x every
0 x effort
582 x forward
2 x inches
0 x moves
0 x pizza
343 x toward
```

基于输出结果我们可以看到单词“forward”大部分时间都被采样（1000次中有582次），但其他token诸如“closer”、“inch”、“toward”有时也会被采样。这意味着，如果我们用generate_and_print_sample函数中的多项式函数代替argmax函数，LLM有时会生成诸如“every effort moves you toward”、“every effort moves you inches”和“every effort moves you closer”而不是“every effort moves you forward”。

我们可以通过温度放缩（temperature scaling）的概念来进一步控制分布和选择过程，所谓的温度放缩实际上只是对logit除以大于0的数这一过程的一个花哨的描述：

```
def softmax_with_temperature(logits, temperature):
scaled_logits = logits / temperature
return torch.softmax(scaled_logits, dim=0)
```

温度大于1会导致token分布更均匀，温度小于1会导致分布更可靠（更清晰或峰值更高）。让我们通过绘制关于原始概率以及使用不同温度值放缩的概率图来说明这一点：

```
temperatures = [1, 0.1, 5] # Original, higher, and lower temperature
scaled_probas = [softmax_with_temperature(next_token_logits, T) for T in temperatures]
x = torch.arange(len(vocab))
bar_width = 0.15
fig, ax = plt.subplots(figsize=(5, 3))
for i, T in enumerate(temperatures):
    rects = ax.bar(x + i * bar_width, scaled_probas[i],
                   bar_width, label=f'Temperature = {T}')
ax.set_ylabel('Probability')
ax.set_xticks(x)
ax.set_xticklabels(vocab.keys(), rotation=90)
ax.legend()
plt.tight_layout()
plt.show()
```

绘制结果如fig-5-14所示。

![fig-5-14温度为1表示词汇表中每个token的未缩放概率分数。将温度降低到 0.1会使分布更加清晰，因此最有可能的token（此处为“正向”）将具有更高的概率分数。反之亦然，将温度提高到5会使分布更加均匀。](https://github.com/datawhalechina/llms-from-scratch-cn/blob/main/Translated_Book/img/fig-5-14.jpg?raw=true)

当温度为1时，将对数除以1然后将其传递给softmax函数来计算概率分数。换言之，使用温度值为1的放缩与不使用任何温度放缩相同。在这种情况下选择的token的概率等同于通过PyTorch中的多项式采样函数获取的原始softmax概率分数。

例如,如figure-5-14所示，当温度设置为1时，大约有60%的时间会选择与“forward”对应的token。

此外如figure-5-14所示，施加非常小的温度（例如0.1）将导致分布的差异性更高，从而使得多项式函数类似于argmax函数一样几乎100%表现为选择可能性最大的token（此处为“正向”）。反之亦然，当温度为5时分布会更均衡使得选择其他token的几率增加。这样可以提高生成文本的多样性但也会导致生成更多无意义的文本。举个例子：温度设置为4会导致大约有4%的时间出现诸如“every effort moves you pizza”这样的文本。

**练习 5.1**

使用 print_sampled_tokens 函数绘制 softmax 概率的采样频率，该频率与figure-5-13 所示的温度成比例。在每种情况下，“pizza”这个词多久抽样一次？你能想出一种更快、更准确的方法来确定“pizza”这个词的采样频率吗？

## 5.3.2 Top-k采样

在前面一节中，我们实现了概率采样方法与温度放缩（temperature scaling）相结合来增加输出结果的多样性。可以看到温度值越高下一个token的概率分布越均衡，由于降低了模型重复选择最可能tokrn的可能性导致产生更具多样性的输出。这种方法允许在生成过程中探索可能性低但也许更有趣和更具创造性的路径。然而该方法也存在一个弊端：该方法有时会导致语法错误或者完全无意义的输出，例如“every effort moves you pizza”。

本节我们将会介绍另一个概念称作top-k采样，当它与概率采样和温度放缩结合时，可以优化文本生成结果。

在top-k采样中我们将抽取的token限制为最有可能的top-k的token，并通过屏蔽概率分数的方式排除所有其他的token，如fig-5-15所示。

![fig-5-15 使用k=3的top-k采样，关注logits最高对应的3个token并在运行softmax函数之前用负无穷大（-inf）屏蔽其他所有的token。这将导致所有非top-k的token概率值归零的概率分布](https://github.com/datawhalechina/llms-from-scratch-cn/blob/main/Translated_Book/img/fig-5-15.jpg?raw=true)

fig-5-15中概述的方法将所有为被选择的logits替换为负无穷大（-inf），这样在计算softmax值时非top-k的token的概率分数为0，其余概率总和为1.（细心的读者或许还记得在第3章第3.5.1节“应用因果注意掩码”中实现的来自因果注意模块的掩码技巧）

我们可以用代码实现fig-5-15中概述的 top-k 过程，如下所示，从选择具有最大 logit值的token开始：

```
top_k = 3
top_logits, top_pos = torch.topk(next_token_logits, top_k)
print("Top logits:", top_logits)
print("Top positions:", top_pos)


前3个token的logit值和token ID（按降序排列）如下所示：

```
Top logits: tensor([6.7500, 6.2800, 4.5100])
Top positions: tensor([3, 7, 0])
```

随后我们运用PyTorch的where函数将低于前3个选项里最低logit值的token的 logit 值设置为负无穷大（-inf）。

```
new_logits = torch.where(
    condition=next_token_logits < top_logits[-1], #A
    input=torch.tensor(float('-inf')), #B
    other=next_token_logits #C
)
print(new_logits)
```

9个token词汇表中下一个token的结果日志如下所示：

```
tensor([4.5100, -inf, -inf, 6.7500, -inf, -inf, -inf, 6.2800, -inf])
```

最后使用softmax函数将这些转换为下一个token概率：

```
topk_probas = torch.softmax(new_logits, dim=0)
print(topk_probas)
```

可以看到这种top-3方法的结果是3个非零概率分数：

```
tensor([0.0615, 0.0000, 0.0000, 0.5775, 0.0000, 0.0000, 0.0000, 0.3610, 0.0000])
```

现在我们可以采用上一节介绍的概率抽样的温度放缩和多项式函数，在这3个非零概率分数中选择下一个token来生成下一个token。下一节中我们将通过修改文本生成函数来实现。

## 5.3.3 修改文本生成函数

前面两节介绍了用于增加LLM生成文本多样性的两个概念：温度采样和top-k采样。本节我们会将这些概念结合来修改我们之前通过LLM来生成文本的generate_simple函数，创造一个新的生成函数：

  **Listing 5.4 具有更多多样性的修改文本生成函数**
```
def generate(model, idx, max_new_tokens, context_size, temperature, top_k=None):
    for _ in range(max_new_tokens): #A
        idx_cond = idx[:, -context_size:]
        with torch.no_grad():
            logits = model(idx_cond)
        logits = logits[:, -1, :]
        if top_k is not None: #B
            top_logits, _ = torch.topk(logits, top_k)
            min_val = top_logits[:, -1]
            logits = torch.where(
                logits < min_val,
                torch.tensor(float('-inf')).to(logits.device),
                logits
            )
        if temperature > 0.0: #C
            logits = logits / temperature
            probs = torch.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
        else: #D
            idx_next = torch.argmax(logits, dim=-1, keepdim=True)
        idx = torch.cat((idx, idx_next), dim=1)
    return idx
```

来看看这个新的生成函数的实际效果：

```
torch.manual_seed(123)
token_ids = generate(
    model=model,
    idx=text_to_token_ids("Every effort moves you", tokenizer),
    max_new_tokens=15,
    context_size=GPT_CONFIG_124M["context_length"],
    top_k=25,
    temperature=1.4
)
print("Output text:\n", token_ids_to_text(token_ids, tokenizer))
```

生成的文本如下所示：


```
Output text:
Every effort moves you stand to work on surprise, a one of us had gone with random
```

可以看到生成的文本与前面在第5.3节开头使用generate_simple 函数生成的文本（以训练集中的一项纪录为例，“Every effort moves you know," was one of the axioms he laid...!”）截然不同。

**练习5.2**

尝试不同的温度和top-k设置。根据您的观察，您能想到需要较低温度和top-k设置的运用吗？反之亦然，您能想到首选需要更高温度和top-k设置的场景吗？（建议在从OpenAI加载预训练权重后，重新回顾本章节末的此练习。

**练习5.3**

生成函数的设置有哪些不同的组合来实现确定性行为（即禁用随机采样使其始终产生接近generate_simple函数的相同输出）？

到目前为止，我们介绍了如何预训练LLM并使用它们来生成文本。本章的最后两节将讨论我们如何保存和加载经过训练的LLM，以及如何从OpenAI加载预训练的权重。